### Note: requires ganache-cli to be installed
### Start ganache-cli as ganache-cli -d --db /data2/web3/ganachedb -i 1234

In [1]:
from web3 import Web3

# Code to compile .sol file and export EVM bytecode and ABI to JSON

In [88]:
contract_path = "RepositoryTracker.sol"
with open(contract_path, "r") as file:
    contract_file = file.read()
# print(contract_file)

In [89]:
from solcx import compile_standard

compiled_solidity = compile_standard(
    {
        "language": "Solidity",
        "sources": {
            "RepositoryTracker.sol": {
                "content": contract_file,
            }
        },
        "settings": {
            "outputSelection": {
                "*": {"*": ["abi", "metadata", "evm.bytecode", "evm.sourceMap"]}
            }
        },
    },
    solc_version="0.8.17",
)

In [15]:
import json

with open("compiled_contract.json", "w") as file:
    json.dump(compiled_solidity, file)

# Get ABI and EVM bytecode

In [16]:
abi = compiled_solidity['contracts']['RepositoryTracker.sol']['RepositoryTracker']['abi']
abi

[{'inputs': [{'internalType': 'string', 'name': 'bafyHash', 'type': 'string'},
   {'internalType': 'string', 'name': 'friendlyName', 'type': 'string'},
   {'internalType': 'address[]', 'name': 'devAddresses', 'type': 'address[]'},
   {'internalType': 'address[]',
    'name': 'auditAddresses',
    'type': 'address[]'}],
  'stateMutability': 'nonpayable',
  'type': 'constructor'},
 {'anonymous': False,
  'inputs': [{'indexed': True,
    'internalType': 'address',
    'name': 'previousOwner',
    'type': 'address'},
   {'indexed': True,
    'internalType': 'address',
    'name': 'newOwner',
    'type': 'address'}],
  'name': 'OwnershipTransferred',
  'type': 'event'},
 {'anonymous': False,
  'inputs': [{'indexed': True,
    'internalType': 'bytes32',
    'name': 'role',
    'type': 'bytes32'},
   {'indexed': True,
    'internalType': 'bytes32',
    'name': 'previousAdminRole',
    'type': 'bytes32'},
   {'indexed': True,
    'internalType': 'bytes32',
    'name': 'newAdminRole',
    'type

In [17]:
bytecode = compiled_solidity['contracts']['RepositoryTracker.sol']['RepositoryTracker']['evm']['bytecode']['object']
# bytecode

# Connect to Test net (Goerli/Ganache local)

In [4]:
ropsten_url = 'https://goerli.infura.io/v3/5953d19885cd43218c5d03280949a537'
ganache_url = 'http://localhost:8545'

w3 = Web3(Web3.HTTPProvider(ganache_url))
w3.isConnected()

True

In [21]:
RepositoryTracker = w3.eth.contract(abi=abi, bytecode=bytecode)
RepositoryTracker

web3._utils.datatypes.Contract

# Deploy Transaction (each repository to be tracked requires a new contract to be deployed)

In [6]:
# attributes to build transaction
goerli_chain = 5
ganache_chain = 1337

metamask_wallet = '0x47f9aEB8F599d320718C92e30C406A893638253E'
metamask_wallet_private_key = ''

ganache_wallet = '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1'
ganache_wallet_private_key = '0x4f3edf983ac636a65a842ce7c78d9aa706d3b113bce9c46f30d7d21715b23b1d'


# change settings here
chainId = ganache_chain
owner_wallet = ganache_wallet
owner_wallet_private_key = ganache_wallet_private_key

nonce = w3.eth.getTransactionCount(owner_wallet)
nonce

35

In [11]:
# Adding fake developers and auditors

developer_wallets = ['0x3E5e9111Ae8eB78Fe1CC3bb8915d5D461F3Ef9A9', '0x28a8746e75304c0780E011BEd21C72cD78cd535E']
developer_private_keys = ['0xe485d098507f54e7733a205420dfddbe58db035fa577fc294ebd14db90767a52', '0xa453611d9419d0e56f499079478fd72c37b251a94bfde4d19872c44cf65386e3']
auditor_wallets =  ['0xACa94ef8bD5ffEE41947b4585a84BdA5a3d3DA6E', '0x1dF62f291b2E969fB0849d99D9Ce41e2F137006e']
auditor_private_keys = ['0x829e924fdf021ba3dbbc4225edfece9aca04b929d6e75613329ca6f1d31c0bb4', '0xb0057716d5917badaf911b193b12b910811c1497b5bada8d7711f758981c3773']

## Build deployment transaction

In [44]:
transaction = RepositoryTracker.constructor('hash', 'reponame', developers, auditors).buildTransaction(
    {
        "gasPrice": w3.eth.gas_price,
        "from": owner_wallet,
        "chainId": chainId,
        "nonce": nonce,
    }
)

## Sign transaction with private key

In [45]:
signed_transaction = w3.eth.account.sign_transaction(transaction, private_key=owner_wallet_private_key)
# signed_transaction

## Perform transaction

In [46]:
tx_hash = w3.eth.send_raw_transaction(signed_transaction.rawTransaction)

# update nonce after each successful transaction: each transaction needs to have a unique nonce
nonce += 1

tx_hash

HexBytes('0x142b20723ab4f7cbbc305a9459ca29aecbf891d80749fd32d23d84c1f589fc04')

In [51]:
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
tx_receipt

AttributeDict({'transactionHash': HexBytes('0x142b20723ab4f7cbbc305a9459ca29aecbf891d80749fd32d23d84c1f589fc04'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0xac170e19aecc19b255c70351266841baead10f89d2bc7cf7867d7e4317510f90'),
 'blockNumber': 30,
 'from': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
 'to': None,
 'gasUsed': 1986362,
 'cumulativeGasUsed': 1986362,
 'contractAddress': '0x86072CbFF48dA3C1F01824a6761A03F105BCC697',
 'logs': [AttributeDict({'logIndex': 0,
   'transactionIndex': 0,
   'transactionHash': HexBytes('0x142b20723ab4f7cbbc305a9459ca29aecbf891d80749fd32d23d84c1f589fc04'),
   'blockHash': HexBytes('0xac170e19aecc19b255c70351266841baead10f89d2bc7cf7867d7e4317510f90'),
   'blockNumber': 30,
   'address': '0x86072CbFF48dA3C1F01824a6761A03F105BCC697',
   'data': '0x',
   'topics': [HexBytes('0x8be0079c531659141344cd1fd0a4f28419497f9722a3daafe3b4186f6b6457e0'),
    HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
    HexBytes('0x00000

## Capture contract address

In [52]:
contract_address = tx_receipt.contractAddress
contract_address

'0x86072CbFF48dA3C1F01824a6761A03F105BCC697'

# Get Contract Instance from Test net

In [53]:
contract_instance = w3.eth.contract(address=contract_address, abi=abi)
contract_instance

## Getter call

In [55]:
contract_instance.functions.getIpfsAddress().call(
    {
        "from": auditor_wallets[1]
    }
)

'hash'

## Simulate setter transaction

In [56]:
contract_instance.functions.setIpfsAddress('settingBafyAddr').call()

ContractLogicError: execution reverted: VM Exception while processing transaction: revert Caller is not a developer!

## Setter call

In [60]:
set_ipfs_addr_transaction = contract_instance.functions.setIpfsAddress('fakeBafyHash').buildTransaction({
        "gasPrice": w3.eth.gas_price,
        "from": developer_wallets[0],
        "chainId": chainId,
        "nonce":  w3.eth.getTransactionCount(developer_wallets[0]),
})

In [61]:
signed_set_ipfs_addr_transaction = w3.eth.account.sign_transaction(set_ipfs_addr_transaction, private_key=developer_private_keys[0])

In [62]:
set_ipfs_tx_hash = w3.eth.send_raw_transaction(signed_set_ipfs_addr_transaction.rawTransaction)
set_ipfs_tx_receipt = w3.eth.wait_for_transaction_receipt(set_ipfs_tx_hash)

set_ipfs_tx_receipt

AttributeDict({'transactionHash': HexBytes('0x0e2dba4595fd255fc540086588af749bfd05281984c30f0b6d42de7b9b737f79'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x42a545fc2aba9fc69c89ef316679a57a01d760a49db14cabc284a34bda739ed2'),
 'blockNumber': 31,
 'from': '0x3E5e9111Ae8eB78Fe1CC3bb8915d5D461F3Ef9A9',
 'to': '0x86072CbFF48dA3C1F01824a6761A03F105BCC697',
 'gasUsed': 29675,
 'cumulativeGasUsed': 29675,
 'contractAddress': None,
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'

In [63]:
contract_instance.functions.getIpfsAddress().call({"from": auditor_wallets[0]})

'fakeBafyHash'

## Test bafyHash with non-owner

In [22]:
try:
    set_ipfs_addr_transaction = contract_instance.functions.setIpfsAddress('fakeBafyHash').buildTransaction({
        "gasPrice": w3.eth.gas_price,
        "from": '0x016Cf2768A5061AeD1e47DF56E9d09Acfe184692',
        "chainId": chainId,
        "nonce": nonce,
    })
    signed_set_ipfs_addr_transaction = w3.eth.account.sign_transaction(set_ipfs_addr_transaction, private_key='0x0f14c58877d8dbc422f158fbf1b654f8068c4f7cdbb8cdbcae2e671c78969713')

    set_ipfs_tx_hash = w3.eth.send_raw_transaction(signed_set_ipfs_addr_transaction.rawTransaction)
    set_ipfs_tx_receipt = w3.eth.wait_for_transaction_receipt(set_ipfs_tx_hash)

    # update nonce after each successful transaction: each transaction needs to have a unique nonce
    nonce += 1

    set_ipfs_tx_receipt
except Exception as e:
    print(type(e))
    # print(e.with_traceback())

<class 'web3.exceptions.ContractLogicError'>


ContractLogicError: execution reverted: VM Exception while processing transaction: revert Ownership Assertion: Caller of the function is not the owner.

# L1 Integration with dGit

## dGit init

In [23]:
! pwd

/data2/web3


In [24]:
#  creating a test repository
%cd /data2/web3/
! rm -rf test-repo
! mkdir test-repo

/data2/web3


In [25]:
# marking dgit as executable
! chmod +x dgit.sh

In [26]:
# initialize git repository, stage changes, and commit
%cd test-repo
! pwd

! ../dgit.sh -c init

! echo "hello world 2" > test.txt
! cat test.txt
! git add .
! git commit -m "test test"

/data2/web3/test-repo
/data2/web3/test-repo
--------------------------------------------------------------------------------
Decentralized-Git-on-IPFS Wrapper
--------------------------------------------------------------------------------
init
Using default password! 

Empty directory detected... creating README.md
hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /data2/web3/test-repo/.git/
[master (root-commit) 7c47aee] Initialized new repository
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 README

In [27]:
# capture bafyHash and friendlyName from .git/description

with open(".git/description", "r") as file:
    bafyHash = file.readline().strip()
    friendlyName = file.readline().strip()

bafyHash, friendlyName

('Qmee8P1DWqeh7ZHy1J1RvsAK55EZv1duT31XPkaH9nLZwz', 'test-repo')

### Create contract with repo info

In [28]:
new_repo_tx = RepositoryTracker.constructor(bafyHash, friendlyName).buildTransaction(
    {
        "gasPrice": w3.eth.gas_price,
        "from": wallet,
        "chainId": chainId,
        "nonce": nonce,
    }
)

new_repo_tx_signed = w3.eth.account.sign_transaction(new_repo_tx, private_key=wallet_private_key)
tx_hash = w3.eth.send_raw_transaction(new_repo_tx_signed.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

nonce += 1

contract_address = tx_receipt.contractAddress

tx_receipt

AttributeDict({'transactionHash': HexBytes('0x7b0fef9cc99ec74e6da2f506e6dc70510fb9dbcce52d347d194d8e6b067900f2'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x90a6816a045099d72c359bee9db1c87954f3e457dae45740431086a831b8fc24'),
 'blockNumber': 27,
 'from': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
 'to': None,
 'gasUsed': 745915,
 'cumulativeGasUsed': 745915,
 'contractAddress': '0xaD888d0Ade988EbEe74B8D4F39BF29a8d0fe8A8D',
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

### Read contract and verify changes

In [29]:
contract_instance = w3.eth.contract(address=tx_receipt.contractAddress, abi=abi)
display(contract_instance.functions.getIpfsAddress().call())
display(contract_instance.functions.getFriendlyName().call())

'Qmee8P1DWqeh7ZHy1J1RvsAK55EZv1duT31XPkaH9nLZwz'

'test-repo'

## dGit push

In [30]:
! touch newfile
! git add .
! git commit -m "test push"
! ../dgit.sh -c push -k "password"

[master 0f02610] test push
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 newfile
--------------------------------------------------------------------------------
Decentralized-Git-on-IPFS Wrapper
--------------------------------------------------------------------------------
push password
Fetching repository from IPFS... 

Saving file(s) to /home/rahul/.tmp/Qmee8P1DWqeh7ZHy1J1RvsAK55EZv1duT31XPkaH9nLZwz.enc
 17.46 KiB / 17.46 KiB [============================================] 100.00% 0s

Decrypting contents... 

Friendly repository name: test-repo

Pushing repository... 

Enumerating objects: 6, done.
Counting objects: 100% (6/6), done.
Delta compression using up to 8 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (5/5), 473 bytes | 473.00 KiB/s, done.
Total 5 (delta 1), reused 0 (delta 0), pack-reused 0
To /home/rahul/.tmp/bare/test-repo
   7c47aee..0f02610  master -> master
 20.15 KiB / 20.15 KiB [=========================================

### Update bafyHash (and friendlyName) on repo contract

In [31]:
# capture bafyHash and friendlyName from .git/description

with open(".git/description", "r") as file:
    bafyHash = file.readline().strip()
    friendlyName = file.readline().strip()

display(bafyHash, friendlyName)

contract_instance = w3.eth.contract(address=contract_address, abi=abi)

update_repo_contract_tx = contract_instance.functions.setIpfsAddress(bafyHash).buildTransaction(
    {
        "gasPrice": w3.eth.gas_price,
        "from": wallet,
        "chainId": chainId,
        "nonce": nonce,
    }
)

update_repo_contract_tx_signed = w3.eth.account.sign_transaction(update_repo_contract_tx, private_key=wallet_private_key)
tx_hash = w3.eth.send_raw_transaction(update_repo_contract_tx_signed.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

nonce += 1

tx_receipt

'QmWQFRsYCegGEuaZMDJgZKWU3KpmATXoctE8Qwrsg76vgh'

'test-repo'

AttributeDict({'transactionHash': HexBytes('0xfaeef4803245f55623d9eaae3c043c8fb4cf1456641fed0d9c9a905239da7f65'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x93b13510e0d9d284f26bf453a40bdbb17ca7defe8c76cf580451dea4abd8e387'),
 'blockNumber': 28,
 'from': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
 'to': '0xaD888d0Ade988EbEe74B8D4F39BF29a8d0fe8A8D',
 'gasUsed': 36249,
 'cumulativeGasUsed': 36249,
 'contractAddress': None,
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'

In [32]:
# verifying changes
# contract_instance = w3.eth.contract(address=tx_receipt.contractAddress, abi=abi)
display(contract_instance.functions.getIpfsAddress().call())
display(contract_instance.functions.getFriendlyName().call())

'QmWQFRsYCegGEuaZMDJgZKWU3KpmATXoctE8Qwrsg76vgh'

'test-repo'

## dGit clone

In [35]:
%cd /data2/web3/
! pwd
! rm -rf test-repo/
! ./dgit.sh -c clone -k "password" -i QmWQFRsYCegGEuaZMDJgZKWU3KpmATXoctE8Qwrsg76vgh

/data2/web3
/data2/web3
--------------------------------------------------------------------------------
Decentralized-Git-on-IPFS Wrapper
--------------------------------------------------------------------------------
clone password QmWQFRsYCegGEuaZMDJgZKWU3KpmATXoctE8Qwrsg76vgh
Fetching repository from IPFS... 

Saving file(s) to /home/rahul/.tmp/QmWQFRsYCegGEuaZMDJgZKWU3KpmATXoctE8Qwrsg76vgh.enc
 20.15 KiB / 20.15 KiB [============================================] 100.00% 0s

Decrypting contents... 

Friendly repository name: test-repo

Cloning repository... 

Cloning into 'test-repo'...
done.

Cleaning up... 

Writing repository tracking information to disk... 



## dGit pull

In [36]:
%cd /data2/web3/test-repo/
! pwd
! ../dgit.sh -c pull -k password -i QmWQFRsYCegGEuaZMDJgZKWU3KpmATXoctE8Qwrsg76vgh

/data2/web3/test-repo
/data2/web3/test-repo
--------------------------------------------------------------------------------
Decentralized-Git-on-IPFS Wrapper
--------------------------------------------------------------------------------
pull password QmWQFRsYCegGEuaZMDJgZKWU3KpmATXoctE8Qwrsg76vgh
Fetching repository from IPFS... 

Saving file(s) to /home/rahul/.tmp/QmWQFRsYCegGEuaZMDJgZKWU3KpmATXoctE8Qwrsg76vgh.enc
 20.15 KiB / 20.15 KiB [============================================] 100.00% 0s

Decrypting contents... 

Friendly repository name: test-repo

Pulling repository... 

From /home/rahul/.tmp/bare/test-repo
 * branch            HEAD       -> FETCH_HEAD
Already up to date.

Cleaning up... 



### TODO: Implement code as .py and integrate with dGit

# L2 Integration with dGit

## Implementing roles

### Trying a realistic use case:
1. Compiling smart contract (v2)

In [220]:
%cd /data2/web3/
contract_path = "RepositoryTracker.sol"
with open(contract_path, "r") as file:
    contract_file = file.read()
# print(contract_file)

from solcx import compile_standard

compiled_solidity = compile_standard(
    {
        "language": "Solidity",
        "sources": {
            "RepositoryTracker.sol": {
                "content": contract_file,
            }
        },
        "settings": {
            "outputSelection": {
                "*": {"*": ["abi", "metadata", "evm.bytecode", "evm.sourceMap"]}
            }
        },
    },
    solc_version="0.8.17",
)

import json

with open("compiled_contract.json", "w") as file:
    json.dump(compiled_solidity, file)

abi = compiled_solidity['contracts']['RepositoryTracker.sol']['RepositoryTracker']['abi']
bytecode = compiled_solidity['contracts']['RepositoryTracker.sol']['RepositoryTracker']['evm']['bytecode']['object']

/data2/web3


In [221]:
RepositoryTracker = w3.eth.contract(abi=abi, bytecode=bytecode)

transaction = RepositoryTracker.constructor('hash', 'reponame', developer_wallets, auditor_wallets).buildTransaction(
    {
        "gasPrice": w3.eth.gas_price,
        "from": owner_wallet,
        "chainId": chainId,
        "nonce":  w3.eth.getTransactionCount(owner_wallet),
    }
)

signed_transaction = w3.eth.account.sign_transaction(transaction, private_key=owner_wallet_private_key)

tx_hash = w3.eth.send_raw_transaction(signed_transaction.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
contract_address = tx_receipt.contractAddress
contract_address

'0x47a2Db5D68751EeAdFBC44851E84AcDB4F7299Cc'

In [186]:
contract_inst = w3.eth.contract(address=contract_address, abi=abi)

2. Attempting to create a new issue

In [188]:
tx = contract_inst.functions.newIssue('new issue', 'check this out', 'bafyyyy').buildTransaction(
    {
        "gasPrice": w3.eth.gas_price,
        "from": auditor_wallets[0],
        "chainId": chainId,
        "nonce":  w3.eth.getTransactionCount(auditor_wallets[0]),
    }
)

s_tx = w3.eth.account.sign_transaction(tx, private_key=auditor_private_keys[0])

tx_hash = w3.eth.send_raw_transaction(s_tx.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
tx_receipt
# TODO: Capture issueId from tx

AttributeDict({'transactionHash': HexBytes('0xdb9753a3083fecb309deaf1334e17e1efcf09c5c8309c174a480bae71cbde0b9'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x867e762f34e416f0c5307bac315a8b2fdff80058b9f550d04fe59aaf1ee538e9'),
 'blockNumber': 142,
 'from': '0xACa94ef8bD5ffEE41947b4585a84BdA5a3d3DA6E',
 'to': '0xc0b3B62DD0400E4baa721DdEc9B8A384147b23fF',
 'gasUsed': 141799,
 'cumulativeGasUsed': 141799,
 'contractAddress': None,
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [189]:
# Check if the issue is created
contract_inst.functions.getIssue(0).call()

('0xACa94ef8bD5ffEE41947b4585a84BdA5a3d3DA6E',
 'new issue',
 'check this out',
 [],
 [],
 'bafyyyy',
 '0x0000000000000000000000000000000000000000',
 [],
 0)

In [190]:
# creating a new comment on the issue
tx = contract_inst.functions.commentOnIssue(issueId=0, comment="this is a comment from auditor 1").buildTransaction(
    {
        "gasPrice": w3.eth.gas_price,
        "from": auditor_wallets[1],
        "chainId": chainId,
        "nonce":  w3.eth.getTransactionCount(auditor_wallets[1]),
    }
)

s_tx = w3.eth.account.sign_transaction(tx, private_key=auditor_private_keys[1])

tx_hash = w3.eth.send_raw_transaction(s_tx.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
tx_receipt

AttributeDict({'transactionHash': HexBytes('0xd97fa352a7a9a1994c2568e44d2cb9f5c7246f49cd8db849cf176c0c723d7771'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x67b4387e6916262c1c5984644cde1abc6eda56dd6aa18ef941ce1448e5c8bf93'),
 'blockNumber': 143,
 'from': '0x1dF62f291b2E969fB0849d99D9Ce41e2F137006e',
 'to': '0xc0b3B62DD0400E4baa721DdEc9B8A384147b23fF',
 'gasUsed': 129730,
 'cumulativeGasUsed': 129730,
 'contractAddress': None,
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [191]:
# read comment
contract_inst.functions.getIssue(0).call()

('0xACa94ef8bD5ffEE41947b4585a84BdA5a3d3DA6E',
 'new issue',
 'check this out',
 ['0x1dF62f291b2E969fB0849d99D9Ce41e2F137006e'],
 ['this is a comment from auditor 1'],
 'bafyyyy',
 '0x0000000000000000000000000000000000000000',
 [],
 0)

In [192]:
# try to update details of the issue
# works only for the person who created the issue!
tx = contract_inst.functions.updateIssueTitle(issueId=0, title="this issue is fake").buildTransaction(
    {
        "gasPrice": w3.eth.gas_price,
        "from": auditor_wallets[0],
        "chainId": chainId,
        "nonce":  w3.eth.getTransactionCount(auditor_wallets[0]),
    }
)

s_tx = w3.eth.account.sign_transaction(tx, private_key=auditor_private_keys[0])

tx_hash = w3.eth.send_raw_transaction(s_tx.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

# trying to assign a dev and update the status of the issue
tx = contract_inst.functions.assignDeveloperToIssue(issueId=0, developerAddress=developer_wallets[0]).buildTransaction(
    {
        "gasPrice": w3.eth.gas_price,
        "from": developer_wallets[1],
        "chainId": chainId,
        "nonce":  w3.eth.getTransactionCount(developer_wallets[1]),
    }
)

s_tx = w3.eth.account.sign_transaction(tx, private_key=developer_private_keys[1])

tx_hash = w3.eth.send_raw_transaction(s_tx.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

# close an issue
tx = contract_inst.functions.closeIssue(issueId=0).buildTransaction(
    # 0: OPEN, 1: ASSIGNED, 2: CLOSED
    {
        "gasPrice": w3.eth.gas_price,
        "from": developer_wallets[1],
        "chainId": chainId,
        "nonce":  w3.eth.getTransactionCount(developer_wallets[1]),
    }
)

s_tx = w3.eth.account.sign_transaction(tx, private_key=developer_private_keys[1])

tx_hash = w3.eth.send_raw_transaction(s_tx.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

display(contract_inst.functions.getIssue(0).call())

('0xACa94ef8bD5ffEE41947b4585a84BdA5a3d3DA6E',
 'this issue is fake',
 'check this out',
 ['0x1dF62f291b2E969fB0849d99D9Ce41e2F137006e'],
 ['this is a comment from auditor 1'],
 'bafyyyy',
 '0x3E5e9111Ae8eB78Fe1CC3bb8915d5D461F3Ef9A9',
 [],
 2)

In [194]:
# adding a label to the issue
tx = contract_inst.functions.addLabelToIssue(issueId=0, label="new la").buildTransaction(
    {
        "gasPrice": w3.eth.gas_price,
        "from": developer_wallets[1],
        "chainId": chainId,
        "nonce":  w3.eth.getTransactionCount(developer_wallets[1]),
    }
)

s_tx = w3.eth.account.sign_transaction(tx, private_key=developer_private_keys[1])

tx_hash = w3.eth.send_raw_transaction(s_tx.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

display(contract_inst.functions.getIssue(0).call())

('0xACa94ef8bD5ffEE41947b4585a84BdA5a3d3DA6E',
 'this issue is fake',
 'check this out',
 ['0x1dF62f291b2E969fB0849d99D9Ce41e2F137006e'],
 ['this is a comment from auditor 1'],
 'bafyyyy',
 '0x3E5e9111Ae8eB78Fe1CC3bb8915d5D461F3Ef9A9',
 ['new label2', 'new la'],
 2)

3. Attempting to create a new pull request

In [172]:
tx = contract_inst.functions.newPullRequest('new PR', 'check this out', 'oldBafy', 'newBafy', ).buildTransaction(
    {
        "gasPrice": w3.eth.gas_price,
        "from": auditor_wallets[1],
        "chainId": chainId,
        "nonce":  w3.eth.getTransactionCount(auditor_wallets[1]),
    }
)

s_tx = w3.eth.account.sign_transaction(tx, private_key=auditor_private_keys[1])

tx_hash = w3.eth.send_raw_transaction(s_tx.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
tx_receipt
# TODO: Capture issueId from tx

AttributeDict({'transactionHash': HexBytes('0x3dc8503423f39bce7cb6bb5adc0e7b3c5bc392d38f9d07828eb318f63d18bac8'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x75573f9685a4d39dfe8f1d7f7975ae4a9db290dd8f2f97ed9a4fb9a6348e11b7'),
 'blockNumber': 133,
 'from': '0x1dF62f291b2E969fB0849d99D9Ce41e2F137006e',
 'to': '0x4cFB3F70BF6a80397C2e634e5bDd85BC0bb189EE',
 'gasUsed': 164137,
 'cumulativeGasUsed': 164137,
 'contractAddress': None,
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [173]:
# Check if the PR is created
contract_inst.functions.getPullRequest(0).call()

('0x1dF62f291b2E969fB0849d99D9Ce41e2F137006e',
 'new PR',
 'check this out',
 [],
 [],
 'oldBafy',
 'newBafy',
 '0x0000000000000000000000000000000000000000',
 [],
 0)

In [174]:
# creating a new comment on the pr
tx = contract_inst.functions.commentOnPullRequest(prId=0, comment="this is a comment from auditor 1").buildTransaction(
    {
        "gasPrice": w3.eth.gas_price,
        "from": auditor_wallets[1],
        "chainId": chainId,
        "nonce":  w3.eth.getTransactionCount(auditor_wallets[1]),
    }
)

s_tx = w3.eth.account.sign_transaction(tx, private_key=auditor_private_keys[1])

tx_hash = w3.eth.send_raw_transaction(s_tx.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
tx_receipt

AttributeDict({'transactionHash': HexBytes('0xf67b7e398f03602001f9a67498d1b35fc9b54cde3d6036d9df505e7d77947078'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x2a66e34cd718d21f9a978f9a419b541de97d8507ba4086de828d8db2428d2d14'),
 'blockNumber': 134,
 'from': '0x1dF62f291b2E969fB0849d99D9Ce41e2F137006e',
 'to': '0x4cFB3F70BF6a80397C2e634e5bDd85BC0bb189EE',
 'gasUsed': 129755,
 'cumulativeGasUsed': 129755,
 'contractAddress': None,
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [175]:
# read comment
contract_inst.functions.getPullRequest(0).call()

('0x1dF62f291b2E969fB0849d99D9Ce41e2F137006e',
 'new PR',
 'check this out',
 ['0x1dF62f291b2E969fB0849d99D9Ce41e2F137006e'],
 ['this is a comment from auditor 1'],
 'oldBafy',
 'newBafy',
 '0x0000000000000000000000000000000000000000',
 [],
 0)

In [176]:
# try to update details of the PR
# works only for the person who created the PR!
tx = contract_inst.functions.updatePullRequestDescription(prId=0, desc="this issue is fake").buildTransaction(
    {
        "gasPrice": w3.eth.gas_price,
        "from": auditor_wallets[1],
        "chainId": chainId,
        "nonce":  w3.eth.getTransactionCount(auditor_wallets[1]),
    }
)

s_tx = w3.eth.account.sign_transaction(tx, private_key=auditor_private_keys[1])

tx_hash = w3.eth.send_raw_transaction(s_tx.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

# trying to assign an auditor and update the status of the PR
tx = contract_inst.functions.assignAuditorToPullRequest(prId=0, auditorAddress=auditor_wallets[1]).buildTransaction(
    {
        "gasPrice": w3.eth.gas_price,
        "from": developer_wallets[1],
        "chainId": chainId,
        "nonce":  w3.eth.getTransactionCount(developer_wallets[1]),
    }
)

s_tx = w3.eth.account.sign_transaction(tx, private_key=developer_private_keys[1])

tx_hash = w3.eth.send_raw_transaction(s_tx.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
tx_receipt

AttributeDict({'transactionHash': HexBytes('0xc6bdd2988b748b81f25c57a77a0a7274e2173f36b45a607ab08b4bc77dc71bf9'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0xbc59b90ad9f8a81f5343d4aacc6453143be5b8a4ac33c77599e42e3d14113bd0'),
 'blockNumber': 136,
 'from': '0x28a8746e75304c0780E011BEd21C72cD78cd535E',
 'to': '0x4cFB3F70BF6a80397C2e634e5bDd85BC0bb189EE',
 'gasUsed': 45105,
 'cumulativeGasUsed': 45105,
 'contractAddress': None,
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [177]:
# trying to approve an PR (by assigned auditor)
tx = contract_inst.functions.approvePullRequest(prId=0).buildTransaction(
    {
        "gasPrice": w3.eth.gas_price,
        "from": auditor_wallets[1],
        "chainId": chainId,
        "nonce":  w3.eth.getTransactionCount(auditor_wallets[1]),
    }
)

s_tx = w3.eth.account.sign_transaction(tx, private_key=auditor_private_keys[1])

tx_hash = w3.eth.send_raw_transaction(s_tx.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
tx_receipt

AttributeDict({'transactionHash': HexBytes('0x3c48ecad42f104097e05d7371d42ab1d2b8f0881933a407b2b48110d7f2df55f'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0xad2fbde7d26dc0e53b7cf53dc0c7a76734a65235fb3d548a8ba73866c84552aa'),
 'blockNumber': 137,
 'from': '0x1dF62f291b2E969fB0849d99D9Ce41e2F137006e',
 'to': '0x4cFB3F70BF6a80397C2e634e5bDd85BC0bb189EE',
 'gasUsed': 48133,
 'cumulativeGasUsed': 48133,
 'contractAddress': None,
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [178]:
# merge the PR
tx = contract_inst.functions.mergePullRequest(prId=0, bafyHash="newBafyHash").buildTransaction(
    {
        "gasPrice": w3.eth.gas_price,
        "from": developer_wallets[1],
        "chainId": chainId,
        "nonce":  w3.eth.getTransactionCount(developer_wallets[1]),
    }
)

s_tx = w3.eth.account.sign_transaction(tx, private_key=developer_private_keys[1])

tx_hash = w3.eth.send_raw_transaction(s_tx.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

display(contract_inst.functions.getPullRequest(0).call())

('0x1dF62f291b2E969fB0849d99D9Ce41e2F137006e',
 'new PR',
 'this issue is fake',
 ['0x1dF62f291b2E969fB0849d99D9Ce41e2F137006e'],
 ['this is a comment from auditor 1'],
 'oldBafy',
 'newBafy',
 '0x1dF62f291b2E969fB0849d99D9Ce41e2F137006e',
 [],
 3)

In [181]:
# adding a label to the PR
tx = contract_inst.functions.addLabelToPullRequest(prId=0, label="new label2").buildTransaction(
    {
        "gasPrice": w3.eth.gas_price,
        "from": developer_wallets[1],
        "chainId": chainId,
        "nonce":  w3.eth.getTransactionCount(developer_wallets[1]),
    }
)

s_tx = w3.eth.account.sign_transaction(tx, private_key=developer_private_keys[1])

tx_hash = w3.eth.send_raw_transaction(s_tx.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

display(contract_inst.functions.getPullRequest(0).call())

('0x1dF62f291b2E969fB0849d99D9Ce41e2F137006e',
 'new PR',
 'this issue is fake',
 ['0x1dF62f291b2E969fB0849d99D9Ce41e2F137006e'],
 ['this is a comment from auditor 1'],
 'oldBafy',
 'newBafy',
 '0x1dF62f291b2E969fB0849d99D9Ce41e2F137006e',
 ['new label', 'new label2'],
 3)

# L2 Test compatibility with dGit

## dGit init

In [280]:
! pwd

/data2/web3


In [281]:
#  creating a test repository
%cd /data2/web3/
! rm -rf test-repo
! mkdir test-repo

/data2/web3


In [282]:
# marking dgit as executable
! chmod +x dgit.sh

In [283]:
# initialize git repository, stage changes, and commit
%cd test-repo
! pwd

! ../dgit.sh -c init

/data2/web3/test-repo
/data2/web3/test-repo
--------------------------------------------------------------------------------
Decentralized-Git-on-IPFS Wrapper
--------------------------------------------------------------------------------
init
Using default password! 

Empty directory detected... creating README.md
hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /data2/web3/test-repo/.git/
[master (root-commit) 5c1ffe8] Initialized new repository
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 README

In [284]:
# capture bafyHash and friendlyName from .git/description

with open(".git/description", "r") as file:
    bafyHash = file.readline().strip()
    friendlyName = file.readline().strip()

bafyHash, friendlyName

('QmUreVXn85jPLav34Yjc8v4HHhmxgWAWa1BoTsceNiPJe9', 'test-repo')

### Create contract with repo info (repo owner does this stuff)

In [285]:
new_repo_tx = RepositoryTracker.constructor(bafyHash, friendlyName, developer_wallets, auditor_wallets).buildTransaction(
    {
        "gasPrice": w3.eth.gas_price,
        "from": owner_wallet,
        "chainId": chainId,
        "nonce": w3.eth.getTransactionCount(owner_wallet),
    }
)

new_repo_tx_signed = w3.eth.account.sign_transaction(new_repo_tx, private_key=owner_wallet_private_key)
tx_hash = w3.eth.send_raw_transaction(new_repo_tx_signed.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

nonce += 1

contract_address = tx_receipt.contractAddress

tx_receipt

AttributeDict({'transactionHash': HexBytes('0x6a675ff94e4d5bc10115b5d290d5a750442b2c53b49ffa3720f9a5e1a2b9edef'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0xd943442b6e2dc940b4d533397933b45920252e7f9ee084843fad69d36f4ac479'),
 'blockNumber': 167,
 'from': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
 'to': None,
 'gasUsed': 5460520,
 'cumulativeGasUsed': 5460520,
 'contractAddress': '0xe97DbD7116D168190F8A6E7beB1092c103c53a12',
 'logs': [AttributeDict({'logIndex': 0,
   'transactionIndex': 0,
   'transactionHash': HexBytes('0x6a675ff94e4d5bc10115b5d290d5a750442b2c53b49ffa3720f9a5e1a2b9edef'),
   'blockHash': HexBytes('0xd943442b6e2dc940b4d533397933b45920252e7f9ee084843fad69d36f4ac479'),
   'blockNumber': 167,
   'address': '0xe97DbD7116D168190F8A6E7beB1092c103c53a12',
   'data': '0x',
   'topics': [HexBytes('0x8be0079c531659141344cd1fd0a4f28419497f9722a3daafe3b4186f6b6457e0'),
    HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
    HexBytes('0x000

### Read contract and verify changes (trying this as an audit/dev account)

In [286]:
contract_instance = w3.eth.contract(address=tx_receipt.contractAddress, abi=abi)
display(contract_instance.functions.getIpfsAddress().call({"from": auditor_wallets[0]}))
display(contract_instance.functions.getFriendlyName().call({"from": developer_wallets[0]}))

'QmUreVXn85jPLav34Yjc8v4HHhmxgWAWa1BoTsceNiPJe9'

'test-repo'

## dGit push

In [287]:
! touch newfile
! git add .
! git commit -m "test push"
! ../dgit.sh -c push -k "password"

[master d9e1593] test push
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 newfile
--------------------------------------------------------------------------------
Decentralized-Git-on-IPFS Wrapper
--------------------------------------------------------------------------------
push password
Fetching repository from IPFS... 

Saving file(s) to /home/rahul/.tmp/QmUreVXn85jPLav34Yjc8v4HHhmxgWAWa1BoTsceNiPJe9.enc
 17.46 KiB / 17.46 KiB [============================================] 100.00% 0s

Decrypting contents... 

Friendly repository name: test-repo

Pushing repository... 

Enumerating objects: 3, done.
Counting objects: 100% (3/3), done.
Delta compression using up to 8 threads
Compressing objects: 100% (2/2), done.
Writing objects: 100% (2/2), 248 bytes | 248.00 KiB/s, done.
Total 2 (delta 0), reused 0 (delta 0), pack-reused 0
To /home/rahul/.tmp/bare/test-repo
   5c1ffe8..d9e1593  master -> master
 18.55 KiB / 18.55 KiB [=========================================

### Create new pull request by dev[0]

In [288]:
# capture bafyHash and friendlyName from .git/description

with open(".git/description", "r") as file:
    bafyHash = file.readline().strip()
    friendlyName = file.readline().strip()

display(bafyHash, friendlyName)

contract_instance = w3.eth.contract(address=contract_address, abi=abi)

tx = contract_instance.functions.newPullRequest(title='adding a new feature', description='check this out', oldBafyHash='QmUreVXn85jPLav34Yjc8v4HHhmxgWAWa1BoTsceNiPJe9', prBafyHash=bafyHash).buildTransaction(
    {
        "gasPrice": w3.eth.gas_price,
        "from": developer_wallets[0],
        "chainId": chainId,
        "nonce":  w3.eth.getTransactionCount(developer_wallets[0]),
    }
)

s_tx = w3.eth.account.sign_transaction(tx, private_key=developer_private_keys[0])

tx_hash = w3.eth.send_raw_transaction(s_tx.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
tx_receipt
# TODO: Capture issueId from tx

'Qme7hNS2etkbF32dCNaJBNUKCsKJEZSd4BHnfbcmZkHZCp'

'test-repo'

AttributeDict({'transactionHash': HexBytes('0x327b45b2eaf84a417b1937eca4c3b90237f3a5f387c59d49c0f9b7078638070b'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x87073fbcf9dac62d42a81269e2f39a8e5c577fa7da4c4eb2148d76953f0a5846'),
 'blockNumber': 168,
 'from': '0x3E5e9111Ae8eB78Fe1CC3bb8915d5D461F3Ef9A9',
 'to': '0xe97DbD7116D168190F8A6E7beB1092c103c53a12',
 'gasUsed': 246947,
 'cumulativeGasUsed': 246947,
 'contractAddress': None,
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [289]:
# contract_instance.functions.newPullRequest(title='adding a new featur2e', description='check this out', oldBafyHash='QmejoFeyuMVcQMJ6VCgw3hasuZwVVvrCMxZS39whPCqSMt', prBafyHash=bafyHash).call({"from": developer_wallets[0]})
contract_instance.functions.getPullRequest(0).call()

('0x3E5e9111Ae8eB78Fe1CC3bb8915d5D461F3Ef9A9',
 'adding a new feature',
 'check this out',
 [],
 [],
 'QmUreVXn85jPLav34Yjc8v4HHhmxgWAWa1BoTsceNiPJe9',
 'Qme7hNS2etkbF32dCNaJBNUKCsKJEZSd4BHnfbcmZkHZCp',
 '0x0000000000000000000000000000000000000000',
 [],
 0)

### Assign an auditor to the PR

In [276]:
# assign an auditor and update the status of the PR
tx = contract_instance.functions.assignAuditorToPullRequest(prId=0, auditorAddress=auditor_wallets[1]).buildTransaction(
    {
        "gasPrice": w3.eth.gas_price,
        "from": developer_wallets[1],
        "chainId": chainId,
        "nonce":  w3.eth.getTransactionCount(developer_wallets[1]),
    }
)

s_tx = w3.eth.account.sign_transaction(tx, private_key=developer_private_keys[1])

tx_hash = w3.eth.send_raw_transaction(s_tx.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

('0x3E5e9111Ae8eB78Fe1CC3bb8915d5D461F3Ef9A9',
 'adding a new feature',
 'check this out',
 [],
 [],
 'QmejoFeyuMVcQMJ6VCgw3hasuZwVVvrCMxZS39whPCqSMt',
 'QmZSfprteRFCGUPs9Q6vJ3QH4exvJCdU7N326BTooiHxBt',
 '0x0000000000000000000000000000000000000000',
 [],
 0)

### dGit pull to accept incoming changes and audit code

In [36]:
%cd /data2/web3/test-repo/
! pwd
! ../dgit.sh -c pull -k password -i QmZSfprteRFCGUPs9Q6vJ3QH4exvJCdU7N326BTooiHxBt

/data2/web3/test-repo
/data2/web3/test-repo
--------------------------------------------------------------------------------
Decentralized-Git-on-IPFS Wrapper
--------------------------------------------------------------------------------
pull password QmWQFRsYCegGEuaZMDJgZKWU3KpmATXoctE8Qwrsg76vgh
Fetching repository from IPFS... 

Saving file(s) to /home/rahul/.tmp/QmWQFRsYCegGEuaZMDJgZKWU3KpmATXoctE8Qwrsg76vgh.enc
 20.15 KiB / 20.15 KiB [============================================] 100.00% 0s

Decrypting contents... 

Friendly repository name: test-repo

Pulling repository... 

From /home/rahul/.tmp/bare/test-repo
 * branch            HEAD       -> FETCH_HEAD
Already up to date.

Cleaning up... 



### Audit code (externally) and approve PR

In [ ]:
# approve an PR (by assigned auditor)
tx = contract_instance.functions.approvePullRequest(prId=0).buildTransaction(
    {
        "gasPrice": w3.eth.gas_price,
        "from": auditor_wallets[1],
        "chainId": chainId,
        "nonce":  w3.eth.getTransactionCount(auditor_wallets[1]),
    }
)

s_tx = w3.eth.account.sign_transaction(tx, private_key=auditor_private_keys[1])

tx_hash = w3.eth.send_raw_transaction(s_tx.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)


### Merge PR

In [ ]:
# merge the PR
tx = contract_instance.functions.mergePullRequest(prId=0, bafyHash=bafyHash).buildTransaction(
    {
        "gasPrice": w3.eth.gas_price,
        "from": developer_wallets[1],
        "chainId": chainId,
        "nonce":  w3.eth.getTransactionCount(developer_wallets[1]),
    }
)

s_tx = w3.eth.account.sign_transaction(tx, private_key=developer_private_keys[1])

tx_hash = w3.eth.send_raw_transaction(s_tx.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
display(contract_instance.functions.getPullRequest(0).call())

## dGit clone

In [279]:
%cd /data2/web3/
! pwd
! rm -rf test-repo/
! ./dgit.sh -c clone -k "password" -i QmWQFRsYCegGEuaZMDJgZKWU3KpmATXoctE8Qwrsg76vgh

/data2/web3
/data2/web3
--------------------------------------------------------------------------------
Decentralized-Git-on-IPFS Wrapper
--------------------------------------------------------------------------------
clone password QmWQFRsYCegGEuaZMDJgZKWU3KpmATXoctE8Qwrsg76vgh
Fetching repository from IPFS... 

Saving file(s) to /home/rahul/.tmp/QmWQFRsYCegGEuaZMDJgZKWU3KpmATXoctE8Qwrsg76vgh.enc
 20.15 KiB / 20.15 KiB [============================================] 100.00% 0s

Decrypting contents... 

Friendly repository name: test-repo

Cloning repository... 

Cloning into 'test-repo'...
done.

Cleaning up... 

Writing repository tracking information to disk... 

